In [ ]:
!pip install transformers torch

# Tokenizers

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")

In [ ]:
raw_inputs =     [
        "I love deep learning!",
        "I hate this so much!",
    ]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[   0,  100,  657, 1844, 2239,  328,    2,    1],
        [   0,  100, 4157,   42,   98,  203,  328,    2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
print('Tokenizer output for "I love deep learning!"')
print(f"Input ids: {inputs['input_ids'][0]}")
print(f"Attention Mask: {inputs['attention_mask'][0]}")
print("-"*100)
print('Tokenizer output for "I hate this so much!"')
print(f"Input ids: {inputs['input_ids'][1]}")
print(f"Attention Mask: {inputs['attention_mask'][1]}")

Tokenizer output for "I love deep learning!"
Input ids: tensor([   0,  100,  657, 1844, 2239,  328,    2,    1])
Attention Mask: tensor([1, 1, 1, 1, 1, 1, 1, 0])
----------------------------------------------------------------------------------------------------
Tokenizer output for "I hate this so much!"
Input ids: tensor([   0,  100, 4157,   42,   98,  203,  328,    2])
Attention Mask: tensor([1, 1, 1, 1, 1, 1, 1, 1])


In [ ]:
tokenizer('I love deep learning!')

{'input_ids': [0, 100, 657, 1844, 2239, 328, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

### Tokenizers Under The Hood

In [ ]:
tokens = tokenizer.tokenize('I love deep learning!')
tokens

['I', 'Ġlove', 'Ġdeep', 'Ġlearning', '!']

In [ ]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
token_ids

[100, 657, 1844, 2239, 328]

In [ ]:
decoded_tokens = tokenizer.decode(token_ids)
decoded_tokens

'I love deep learning!'

In [ ]:
model_prepped_ids = tokenizer.prepare_for_model(token_ids)
model_prepped_ids

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': [0, 100, 657, 1844, 2239, 328, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

# Models

# Pipeline

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I love deep learning!",
        "I hate this so much!",
    ]
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998645782470703},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [ ]:
text_generator = pipeline("text-generation")

text_generator([
    'I went to the store to buy',
    'When two objects in space get close to each other'
])

No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[[{'generated_text': "I went to the store to buy some.\n\nI went over to that one, I thought, and after a few tries, came out with, as I've said before, a couple more. They were super-light and thick. The"}],
 [{'generated_text': 'When two objects in space get close to each other with a beam, if the beam comes to rest on the left side instead of just the right side, or at least close to both sides, this is called the "lightfall." When the laser'}]]

In [ ]:
summarizer = pipeline("summarization")

summarizer([
    """A Fibonacci heap is a collection of trees satisfying the min-heap property. It allows faster amortized time for many operations than binary or binomial heaps.
    Trees in a Fibonacci heap can have any shape, which facilitates efficient operations. Lazy strategies are employed: node removals and consolidations are delayed until
    absolutely necessary (like during an extract-min operation). The main advantage lies in decreasing a key and merging two heaps, which are constant and amortized
    constant time, respectively. Nodes have a "mark" indicating if they've lost a child since the last time they were made a child of another node, assisting in
    restructuring during operations."""
])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'summary_text': ' A Fibonacci heap is a collection of trees satisfying the min-heap property . It allows faster amortized time for many operations than binary or binomial heaps . Nodes have a "mark" indicating if they\'ve lost a child since the last time they were made a child of another node .'}]

## Accessing Pretrained models

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [ ]:
inputs = tokenizer('I love deep learning', return_tensors='pt')
inputs

{'input_ids': tensor([[ 101, 1045, 2293, 2784, 4083,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [ ]:
outputs = model(**inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-4.1975,  4.4937]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

## Model Embeddings

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [ ]:
inputs = tokenizer('I love deep learning!', padding=True, truncation=True, return_tensors='pt')

outputs = model(**inputs)

print(outputs.last_hidden_state.shape) # the token embeddings

torch.Size([1, 7, 768])


In [ ]:
# to get the full context vector for the sequence
context_vectors = outputs.last_hidden_state.mean(dim=1)
context_vectors.shape

torch.Size([1, 768])

## Accessing Model Config & Creating Custom Models

In [ ]:
from transformers import GPT2Config, GPT2Model

# Building the config
config = GPT2Config()

In [ ]:
print(config)

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.38.2",
  "use_cache": true,
  "vocab_size": 50257
}



In [ ]:
# Building the model from the config
gpt_model = GPT2Model(config)

# Saving Models

In [ ]:
gpt_model.save_pretrained('gpt2_model')